## Import Library

In [ ]:
import pandas as pd
import numpy as np

## Data preprocess

In [ ]:
# Load the training data from Kaggle
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

# Split the data into features (X) and labels (y)
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']

# Split the data into features (X) and labels (y)
X_test = test_data

## Library from scratch
- GaussianNB

In [ ]:
class MultinomialNB:
    def __init__(self, alpha=1.0):
        self.alpha = alpha  # Laplace smoothing parameter
        self.class_prior_ = None
        self.feature_log_prob_ = None

    def fit(self, X, y):
        # Calculate class priors
        classes, class_counts = np.unique(y, return_counts=True)
        self.class_prior_ = class_counts / len(y)

        # Calculate conditional probabilities using Laplace smoothing
        num_classes = len(classes)
        num_features = X.shape[1]
        self.feature_log_prob_ = np.zeros((num_classes, num_features))

        for i, c in enumerate(classes):
            class_mask = (y == c)
            class_count = class_counts[i]
            term_counts = X[class_mask].sum(axis=0)
            self.feature_log_prob_[i] = np.log((term_counts + self.alpha) / (class_count + self.alpha * num_features))

    def predict(self, X):
        # Calculate the log probability of each class for each sample
        log_probs = np.dot(X, self.feature_log_prob_.T) + np.log(self.class_prior_)

        # Select the class with the highest log probability
        return np.argmax(log_probs, axis=1)

## Prediction

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Create a DataFrame for the submission
submission = pd.DataFrame({'ImageId': range(1, len(y_pred) + 1), 'Label': y_pred})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)